In [2]:
import altair as alt
import pandas as pd

In [3]:
!gdown 1DiFrA2DjLkkI8crlu92E0qUzKfHe8SH_
!gdown 1htBV5ebQlryLxhEKauhq0CbFXhqdRfmc
!gdown 1h09nzVuu2Jh69QdnnJ7Tp9_O2p1X4OAi

Downloading...
From: https://drive.google.com/uc?id=1DiFrA2DjLkkI8crlu92E0qUzKfHe8SH_
To: /content/indice_IPC.csv
100% 2.60k/2.60k [00:00<00:00, 3.68MB/s]
Downloading...
From: https://drive.google.com/uc?id=1htBV5ebQlryLxhEKauhq0CbFXhqdRfmc
To: /content/Ponderacion.csv
100% 357/357 [00:00<00:00, 479kB/s]
Downloading...
From: https://drive.google.com/uc?id=1h09nzVuu2Jh69QdnnJ7Tp9_O2p1X4OAi
To: /content/Apertura IPC.csv
100% 42.3k/42.3k [00:00<00:00, 48.9MB/s]


In [4]:
Indice = pd.read_csv('indice_IPC.csv', delimiter=';')
Ponderacion = pd.read_csv('Ponderacion.csv', delimiter=';')
Apertura = pd.read_csv('Apertura IPC.csv', delimiter=';')

In [5]:
Apertura

,Fecha,Concepto,Valor,Valor Ponderado,Variacion indice,Varacion indice ponderado,Variación porcentual
0,2018-01-01,Alimentos,99.37,19.1797,0.00,0.0000,0.0000
1,2018-02-01,Alimentos,99.04,19.1160,-0.33,-0.0637,-0.0033
2,2018-03-01,Alimentos,99.16,19.1392,0.12,0.0232,0.0012
3,2018-04-01,Alimentos,99.54,19.2125,0.38,0.0733,0.0038
4,2018-05-01,Alimentos,99.34,19.1739,-0.20,-0.0386,-0.0020
...,...,...,...,...,...,...,...
643,2022-02-01,Vivienda y servicios,116.70,17.3033,1.00,0.1483,0.0086
644,2022-03-01,Vivienda y servicios,117.86,17.4753,1.16,0.1720,0.0099
645,2022-04-01,Vivienda y servicios,119.07,17.6547,1.21,0.1794,0.0103
646,2022-05-01,Vivienda y servicios,120.30,17.8371,1.23,0.1824,0.0103


In [6]:
def get_chart(data):
    hover = alt.selection_single(
        fields=["Periodo"],
        nearest=True,
        on="mouseover",
        empty="none",
    )

    lines = (
        alt.Chart(data, title="Evolución de Indices IPC")
        .mark_line()
        .encode(
            x="Periodo",
            y="Indice",
            color="Tipo",
            # strokeDash="symbol",
        )
    )

    # Draw points on the line, and highlight based on selection
    points = lines.transform_filter(hover).mark_circle(size=65)

    # Draw a rule at the location of the selection
    tooltips = (
        alt.Chart(data)
        .mark_rule()
        .encode(
            x="yearmonthdate(Periodo)",
            y="Indice",
            opacity=alt.condition(hover, alt.value(0.3), alt.value(0)),
            tooltip=[
                alt.Tooltip("Periodo", title="Periodo"),
                alt.Tooltip("Indice", title="Indice"),
            ],
        )
        .add_selection(hover)
    )

    return (lines + points + tooltips).interactive()
    
    

# Original time series chart. Omitted `get_chart` for clarity
chart = get_chart(Indice)

# Input annotations
ANNOTATIONS = [
    ("Mar 01, 2019", "Inicio Pandemia en Chile"),
]

# Create a chart with annotations
annotations_Indice = pd.DataFrame(ANNOTATIONS, columns=["Periodo", "Tipo"])
annotations_Indice.Periodo = pd.to_datetime(annotations_Indice.Periodo)
annotations_Indice["Indice"] = 0
annotation_layer = (
    alt.Chart(annotations_Indice)
    .mark_text(size=15, text="O", dx=-14, dy=10, align="center")
    .encode(
        x="Periodo:T",
        y=alt.Y("Indice:Q"),
        tooltip=["Tipo"],
    )
    .interactive()
)

# Display both charts together
#alt.altair_chart((chart + annotation_layer).interactive(), use_container_width=True)
chart

alt.LayerChart(...)

In [7]:
selection = alt.selection_multi(fields=['Concepto'], bind='legend')

alt.Chart(Apertura).mark_area().encode(
    alt.X('Fecha:T', axis=alt.Axis(domain=False, format='%Y', tickSize=0)),
    alt.Y('sum(Valor Ponderado):Q', stack='zero', title='Indice Ponderado'),
    alt.Color('Concepto:N', scale=alt.Scale(scheme='category20b')),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).add_selection(
    selection
).properties(
    width=900
)

alt.Chart(...)

In [8]:
highlight = alt.selection(type='single', on='mouseover',
                          fields=['Concepto'], nearest=True)

base = alt.Chart(Apertura).encode(
    x='Fecha:T',
    y='Variación porcentual:Q',
    color='Concepto:N'
)

points = base.mark_circle().encode(
    opacity=alt.value(0)
).add_selection(
    highlight
).properties(
    width=600
)

lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1), alt.value(3))
)

points + lines

alt.LayerChart(...)

In [15]:
base = alt.Chart(Ponderacion).encode(
    theta=alt.Theta("Ponderacion:Q", stack=True), color=alt.Color("Descripción:N")
).transform_calculate(
    emoji= "{'Alimentos': '🍞', 'Alcohol y tabaco': '🍷', 'Vestuario': '👚', 'Vivienda y servicios': '🏡',  'Equipamiento vivienda': '🛠', 'Salud': '🏥',  'Transporte': '🚌', 'Comunicaciones': '📱', 'Cultura': '🎭', 'Educación': '📚', 'Restaurantes y hoteles': '🏨', 'Bienes y servicios diversos': '📦'}[datum.Descripción]"
)

pie = base.mark_arc(outerRadius=150).properties(
    width=600,height=500
)
text = base.mark_text(radius=170, size=20).encode(text="Porcentaje:N")
text2 = base.mark_text(radius=210, size=40).encode(text="emoji:N")

pie + text + text2

alt.LayerChart(...)

In [10]:
Ponderacion

,Descripción,Ponderacion,Porcentaje
0,Alimentos,0.1930,19%
1,Alcohol y tabaco,0.0478,5%
2,Vestuario,0.0351,4%
3,Vivienda y servicios,0.1483,15%
4,Equipamiento vivienda,0.0652,7%
5,Salud,0.0777,8%
6,Transporte,0.1312,13%
7,Comunicaciones,0.0545,5%
8,Cultura,0.0659,7%
9,Educación,0.0660,7%
